In [3]:
%pylab inline
%load_ext autoreload
%autoreload 2
import seaborn as sn

Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys,os
#sys.path.append('..')
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm

import datetime
import pandas as pd
import numpy as np
import scipy

In [17]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)

#INSTRUMENTS_LIST = ['ES', 'CL', 'NG', 'ZN', 'ZS', 'ZW', 'ZC', '6E', 'CC']
# INSTRUMENTS_LIST = ['CL']

#exo_filter = 'neutralOnly'     # All 
#exo_filter = 'ZC'  # ES only
exo_filter = 'ContFut'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

# Do quick sanity checks
for exo in sorted(storage.exo_list(exo_filter)):
    #print(exo)
    exo_df, exo_dict = storage.load_series(exo)
    
    if len(exo_df) < 200:
        print("{0} [NODATA DataLen: {1}]".format(exo, len(exo_df)))
        #storage.delete_exo(exo)
    elif (datetime.datetime.now() - exo_df.index[-1]).days > 4:
        print("{0} [DELAYED: LastDate: {1}]".format(exo, exo_df.index[-1]))
        #storage.delete_exo(exo)
    else:
        print("{0} [OK]".format(exo))

6A_ContFut [OK]
6B_ContFut [OK]
6C_ContFut [OK]
6E_ContFut [OK]
6J_ContFut [OK]
AL1_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
AW6_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
B6L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
CC_ContFut [OK]
CL_ContFut [OK]
DC_ContFut [OK]
E4L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
ES_ContFut [OK]
GC_ContFut [OK]
HE_ContFut [OK]
HG_ContFut [OK]
HO_ContFut [OK]
KC_ContFut [DELAYED: LastDate: 2016-07-11 00:00:00]
L3L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
LBS_ContFut [OK]
LE_ContFut [OK]
N9L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
NG_ContFut [OK]
NQ_ContFut [OK]
R7L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
RB_ContFut [OK]
SB_ContFut [OK]
SI_ContFut [OK]
V3L_ContFut [DELAYED: LastDate: 2018-09-28 00:00:00]
VX_ContFut [OK]
XAB_ContFut [OK]
XAE_ContFut [OK]
XAF_ContFut [OK]
XAI_ContFut [OK]
XAK_ContFut [OK]
XAP_ContFut [OK]
XAU_ContFut [OK]
XAV_ContFut [OK]
XAY_ContFut [OK]
ZB_ContFut [OK]
ZC_ContFut [OK]
ZF

### Delete EXO

In [18]:
# storage.delete_exo('VX_ContFut')

# Single EXO analysis

In [19]:
EXO_NAME = 'ES_ContFut' ###
exo_df, exo_dict = storage.load_series(EXO_NAME)

In [20]:
es = exo_df['exo'].to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/es.csv')

In [21]:
pwd

'/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas'

In [22]:
es = pd.DataFrame(pd.read_csv('es.csv',date_parser=True))
es.columns = [['date','equity']]
es.index = es['date']
es.index = pd.to_datetime(es.index)
del es['date']

In [23]:
es.to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/es.csv')

In [25]:
es_w_deltas = exo_df[['exo','delta']].to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/es_w_deltas.csv')

In [26]:
es_w_deltas = pd.DataFrame(pd.read_csv('es_w_deltas.csv',date_parser=True))
es_w_deltas.columns = [['date','equity', 'delta']]
es_w_deltas.index = es_w_deltas['date']
es_w_deltas.index = pd.to_datetime(es_w_deltas.index)
del es_w_deltas['date']

In [28]:
es_w_deltas.to_csv('/home/tmqr/v1/notebooks/PAM Enhanced/S&P Alphas/es_w_deltas.csv')

In [15]:
es_w_deltas

,equity,delta
date,,
2011-06-01,0.0,1.0
2011-06-02,-62.5,1.0
2011-06-03,-687.5,1.0
2011-06-06,-1400.0,1.0
2011-06-07,-1350.0,1.0
2011-06-08,-1787.5,1.0
2011-06-09,-1237.5,1.0
2011-06-10,-2162.5,1.0
2011-06-13,-2200.0,1.0
